# Wetlab transfer
Author: Famke Baeuerle

This notebook serves as collection of changes to the models that came from wetlab data and expertise.

In [ ]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd

## Missing exchanges

So far none of the models sodium uptake reactions. This makes no sense biologically and thus we will add these components to the models.

Nickel, 34dhbz and biotin are also missing. We want to add exchanges for all of those and add 34dhbz as extracellular metabolite if its missing completely.

In [37]:
def add_and_annotate_exchange(all_models, modelpaths, metabolite):
    for model, path in zip(all_models, modelpaths):
        try:
            model.add_boundary(model.metabolites.get_by_id(metabolite), type="exchange")
            cobra.io.write_sbml_model(model, path)
            model_annot = rg.io.load_model_libsbml(path)
            rea = model_annot.getReaction(f'R_EX_{metabolite}')
            rg.cvterms.add_cv_term_reactions(f'EX_{metabolite}', 'BIGG', rea)
            rg.io.write_to_file(model_annot, path)
            print(f'EX_{metabolite} added to {model.id}.')
        except (ValueError):
            print(f'EX_{metabolite} already exists in {model.id}.')

### 1. Sodium

Sodium is available as extracelullar metabolite in all models but not exchange reaction is given.

In [ ]:
modelpaths = [
        '../../models/Cstr_TS.xml', 
        '../../models/Cstr_1197.xml',
        '../../models/Cstr_1115.xml',
        '../../models/Cstr_1116.xml',
        '../../models/Cstr_KC-Na-01.xml',
        ]

all_models = [rg.io.load_model_cobra(path) for path in modelpaths]

Look at reactions that use extracellular sodium:

In [ ]:
for rea in all_models[4].metabolites.get_by_id('na1_e').reactions:
    print(rea)

In [ ]:
add_and_annotate_exchange(all_models, modelpaths, 'na1_e')

### 2. Nickel

In [29]:
for model in all_models:
    try:
        print(model.metabolites.get_by_id('ni2_e'))
    except (KeyError):
        print(f'Not in {model.id}')

ni2_e
ni2_e
ni2_e
ni2_e
ni2_e


We need to add extracellular Nickel to the model of strain KC-Na-01.

In [30]:
all_models[0].metabolites.get_by_id('ni2_e')

Metabolite identifier,ni2_e
Name,Nickel
Memory address,0x07f82a99044f0
Formula,Ni
Compartment,e
In 3 reaction(s),"NI2tex, NIabc, EX_ni2_e"


In [ ]:
all_models[4]

In [ ]:
ni2_e = cobra.Metabolite('ni2_e', formula='Ni', name='Nickel', compartment='e')
all_models[4].metabolites.add(ni2_e)

In [31]:
add_and_annotate_exchange(all_models, modelpaths, 'ni2_e')

EX_na1_e already exists in fda_1054
EX_na1_e already exists in fda_1197
EX_na1_e already exists in fda_1115
EX_na1_e already exists in fda_1116
EX_na1_e already exists in KC_Na_01


### 3. Biotin

In [32]:
for model in all_models:
    try:
        print(model.metabolites.get_by_id('btn_e'))
    except (KeyError):
        print(f'Not in {model.id}')

btn_e
btn_e
Not in fda_1115
Not in fda_1116
Not in KC_Na_01


We need to add extracellular biotin to 1115 (Cstr_16), 1116 (Cstr_17) and KC-Na-01 (Cstr_KC-Na-01).

In [33]:
all_models[0].metabolites.get_by_id('btn_e')

Metabolite identifier,btn_e
Name,Biotin
Memory address,0x07f829929af70
Formula,C10H15N2O3S
Compartment,e
In 4 reaction(s),"BTNt2i, EX_btn_e, BTNTe, BTNt"


In [35]:
btn_e = cobra.Metabolite('btn_e', formula='C10H15N2O3S', name='Biotin', compartment='e')
all_models[2].metabolites.add(btn_e)
all_models[3].metabolites.add(btn_e)
all_models[4].metabolites.add(btn_e)

In [38]:
add_and_annotate_exchange(all_models, modelpaths, 'btn_e')

EX_btn_e already exists in fda_1054.
EX_btn_e already exists in fda_1197.
EX_btn_e already exists in fda_1115.
EX_btn_e already exists in fda_1116.
Modified model written to ../../models/Cstr_KC-Na-01.xml
EX_btn_e added to KC_Na_01.
